In [1]:
import huggingface_hub
!git config --global credential.helper store
huggingface_hub.login("", add_to_git_credential=True)

/Users/Dagobert/miniforge3/envs/th24/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (osxkeychain,store).
Your token has been saved to /Users/Dagobert/.cache/huggingface/token
Login successful


In [2]:
import argilla as rg

client = rg.Argilla(
    api_url="https://dagobert42-speech-text-classification.hf.space",
    api_key=""
    )

In [3]:
task = """Die Aufgabe besteht darin, die Beispielsätze danach zu kategorisieren, ob diese ursprünglich in einem Podcast gesprochen oder in einem Artikel oder Blog geschrieben worden sind.

**Wichtig:**
Bitte lies dir zunächst einmal gründlich die Richtlinien/Guidelines am Ende dieser Seite durch."""
guide = """Vielen Dank, dass ihr euch Zeit nehmt, an dieser Umfrage teilzunehmen!

In meiner Masterarbeit geht es um Unterschiede zwischen gesprochener und geschriebener Sprache. Dieser Datensatz enthält Textbeispiele aus Podcast-Transkripten, Zeitungsartikeln und Blogs. Als grobes Ziel sollte jeder Teilnehmer der Umfrage 50 Beispiele kategorisieren, was voraussichtlich um die 10 Minuten dauert.

Es hilft natürlich auch viel, wenn ihr es schafft, mehr als 50 Beispiele zu kategorisieren. Bitte nehmt euch dabei aber immer in Ruhe Zeit. Geratene oder hastig gegebene Antworten verfälschen nur das Gesamtergebnis.

Alle Teilnehmer können den Datensatz parallel bearbeiten. Ein Textbeispiel ist fertig bearbeitet, sobald mindestens drei verschiedene Teilnehmer ihm ein Label gegeben haben. Mit einem Klick auf den Balken können der persönliche und gesamtheitliche Fortschritt eingesehen werden.

Es gibt keine falsche oder richtige Vorgehensweise bei dieser Aufgabe. Es ist gut möglich, dass sich die Textbeispiele nicht immer klar einordnen lassen. Versucht also einfach, die Beispiele so gut es geht zuzuordnen."""
settings = rg.Settings(
    guidelines=guide,
    fields=[
        rg.TextField(
            name="task",
            title="Aufgabe",
            use_markdown=True,
        ),
        rg.TextField(
            name="example",
            title="Beispielsatz",
            use_markdown=False,
        )
    ],
    questions=[
        rg.LabelQuestion(
            name="label_question",
            title="In welche Kategorie passt dieser Satz am besten?",
            labels=["gesprochen", "geschrieben"],
        )
    ],
    metadata=[
        rg.TermsMetadataProperty(
            name="original_label",
            options=["gesprochen", "geschrieben"],
            visible_for_annotators=False
        )
    ],
    distribution=rg.TaskDistribution(min_submitted=3)
)

In [4]:
dataset = rg.Dataset(
    name=f"Klassifizierung Deutscher Sprachtexte",
    workspace="argilla",
    settings=settings,
    client=client,
)
dataset.create()

Dataset(id=UUID('deb4cedc-c998-4e16-81b4-d97f22103866') inserted_at=datetime.datetime(2024, 8, 26, 8, 28, 14, 548533) updated_at=datetime.datetime(2024, 8, 26, 8, 28, 15, 843264) name='Klassifizierung Deutscher Sprachtexte' status='ready' guidelines='Vielen Dank, dass ihr euch Zeit nehmt, an dieser Umfrage teilzunehmen!\n\nIn meiner Masterarbeit geht es um Unterschiede zwischen gesprochener und geschriebener Sprache. Dieser Datensatz enthält Textbeispiele aus Podcast-Transkripten, Zeitungsartikeln und Blogs. Als grobes Ziel sollte jeder Teilnehmer der Umfrage 50 Beispiele kategorisieren, was voraussichtlich um die 10 Minuten dauert.\n\nEs hilft natürlich auch viel, wenn ihr es schafft, mehr als 50 Beispiele zu kategorisieren. Bitte nehmt euch dabei aber immer in Ruhe Zeit. Geratene oder hastig gegebene Antworten verfälschen nur das Gesamtergebnis.\n\nAlle Teilnehmer können den Datensatz parallel bearbeiten. Ein Textbeispiel ist fertig bearbeitet, sobald mindestens drei verschiedene Tei

In [5]:
from datasets import load_from_disk

data = load_from_disk("../data/hf_datasets/german_speech_text_recognition")

task_descriptions = [task]*len(data["test"]["label"])
data = data["test"].add_column("task_desc", task_descriptions)

dataset.records.log(
    records=data,
    mapping={
        "task_desc": "task",
        "text": "example",
        "label": "original_label"
        }
    )

/Users/Dagobert/miniforge3/envs/th24/lib/python3.12/site-packages/argilla/records/_mapping/_mapper.py:89: UserWarning: Keys ['text', 'label', '__index_level_0__', 'task_desc'] in data are not present in the mapping and will be ignored.
  warnings.warn(f"Keys {unknown_keys} in data are not present in the mapping and will be ignored.")
Sending records...: 5batch [00:03,  1.37batch/s]                    


DatasetRecords(Dataset(id=UUID('deb4cedc-c998-4e16-81b4-d97f22103866') inserted_at=datetime.datetime(2024, 8, 26, 8, 28, 14, 548533) updated_at=datetime.datetime(2024, 8, 26, 8, 28, 15, 843264) name='Klassifizierung Deutscher Sprachtexte' status='ready' guidelines='Vielen Dank, dass ihr euch Zeit nehmt, an dieser Umfrage teilzunehmen!\n\nIn meiner Masterarbeit geht es um Unterschiede zwischen gesprochener und geschriebener Sprache. Dieser Datensatz enthält Textbeispiele aus Podcast-Transkripten, Zeitungsartikeln und Blogs. Als grobes Ziel sollte jeder Teilnehmer der Umfrage 50 Beispiele kategorisieren, was voraussichtlich um die 10 Minuten dauert.\n\nEs hilft natürlich auch viel, wenn ihr es schafft, mehr als 50 Beispiele zu kategorisieren. Bitte nehmt euch dabei aber immer in Ruhe Zeit. Geratene oder hastig gegebene Antworten verfälschen nur das Gesamtergebnis.\n\nAlle Teilnehmer können den Datensatz parallel bearbeiten. Ein Textbeispiel ist fertig bearbeitet, sobald mindestens drei v